In [1]:
import time
import json
import sys
import os
import glob
import re
import logging
from collections import deque

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer


CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [2]:
fxc = FuncXClient()
# fxs = FuncXSerializer() # (de)serialize the funcx codes

In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    import logging
    from subprocess import PIPE
    
    rec_max_qsize = data['max_qsize']
    rec_ngpu = data['rec_ngpu']
    wid = data['wid']
    dataset_name = data['dataset_name']
    
    log_file_name = "/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/logs/funcx-ptycho-{}-w{}-q{}-g{}.log".format(dataset_name, wid, rec_max_qsize, rec_ngpu)
    
    logging.basicConfig(filename=log_file_name,
                        filemode='a',
                        format='%(asctime)s %(levelname)s %(message)s',
                        level=logging.INFO,
                        datefmt='%Y-%m-%d %H:%M:%S')

    
    
    logging.info("Starting ptycho funcx function.")
    
    remote_log_file_name = "/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/logs/tike-wf-{}-w{}-q{}-g{}.log".format(dataset_name, wid, rec_max_qsize, rec_ngpu)
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #recon. script parameters
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    rec_overwrite = '--overwrite' if (('rec_overwrite' in data) and data['rec_overwrite']) else ''
    rec_auto_pin = '--auto-pin' if (('rec_auto_pin' in data) and data['rec_auto_pin']) else ''
    rec_gpu_id = data['rec_gpu_id']
    rec_log_filename = remote_log_file_name
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' {rec_overwrite} {rec_auto_pin} --gpu-id={rec_gpu_id} --folder='{ofpath}' --wf-max-qsize={rec_max_qsize} --log-file='{rec_log_filename}'"
    logging.info(f"Running command: {cmd}")
    
    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

b6445d5e-c248-4b60-bb49-9c03184ea660


In [4]:
def get_folder_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))


def get_file_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

569b1da5-5c52-4ad2-b2af-0c8577f31a20
a13f13eb-4f02-422e-bd18-eca682ad1c40


In [5]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 1800,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 3600,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 1800,
      "End": True
    },
  }
}

In [6]:
prisma_fx_endpoint = '4bf59543-3398-42d2-9416-c628e9f5635f'

src_wf_root_path = '/prisma-data1/bicer/workflow'
src_input_folder_prefix = "input"
src_output_folder_prefix = "output"

dest_wf_root_path = '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate'
dest_input_folder_prefix = "input"
dest_output_folder_prefix = "output"


src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
rid = fxc.run(src_input_folder_paths_regex, 
              endpoint_id=prisma_fx_endpoint, 
              function_id=fx_func_get_folder_paths_uuid)
src_input_folder_paths = fxc.get_result(rid)

src_output_folder_paths = []
dest_output_folder_paths = []
dest_input_folder_paths = []
for src_input_folder_path in src_input_folder_paths:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
    src_output_folder_paths.append(src_output_folder_path)
    dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
    dest_input_folder_paths.append(dest_input_folder_path)
    dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
    dest_output_folder_paths.append(dest_output_folder_path)
    
    

# src_input_folder_paths: diffraction patterh files to be processed @ APS
# src_output_folder_paths: folders for reconstrcuted images after processing @ APS
# dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
# dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

for (src_input_folder_path, src_output_folder_path, dest_input_folder_path, dest_output_folder_path ) in zip(src_input_folder_paths, src_output_folder_paths, dest_input_folder_paths, dest_output_folder_paths):
    print(f"Source input folder: {src_input_folder_path}; Source output folder: {src_output_folder_path}")
    print(f"Dest. input folder: {dest_input_folder_path}; Dest. output folder: {dest_output_folder_path}")
    print()

Source input folder: /prisma-data1/bicer/workflow/input/scan192; Source output folder: /prisma-data1/bicer/workflow/output/192
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/192; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/192

Source input folder: /prisma-data1/bicer/workflow/input/scan193; Source output folder: /prisma-data1/bicer/workflow/output/193
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/193; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/193

Source input folder: /prisma-data1/bicer/workflow/input/scan194; Source output folder: /prisma-data1/bicer/workflow/output/194
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/194; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/194

Source input folder: /prisma-data1/bicer/workflow/input/scan195; Source output folder

In [7]:
theta_fx_endpoint = '2ab22e1f-4cf1-47e9-a40c-dd8c58b41d73' #theta-ptycho-8w-1n

#src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' #'hostel' #aps/workstation
#dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN # 'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #'alcf#dtn_mira'

# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/tike-pinned-ptycho-recon.py'
python_path = "/home/bicer/projects/tyler/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 1
rec_upd_pos = False
rec_niter = 50
rec_output_freq = 50
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 1
rec_use_mpi = False
rec_overwrite = True
rec_auto_pin = True


dataset_name = "catalyst"
nnodes = 8
nworkers_per_node = 8
MAX_QSIZE = nnodes * nworkers_per_node


flow_inputs = []
gcounter = 0
for (src_input_folder_path, src_output_folder_path, 
     dest_input_folder_path, dest_output_folder_path ) in zip(
    src_input_folder_paths, src_output_folder_paths, 
    dest_input_folder_paths, dest_output_folder_paths):
    
    rec_gpu_id = gcounter%nworkers_per_node
    
    flow_input = {
        "input": {
            "source_endpoint": f"{src_endpoint}",
            "source_path": f"{src_input_folder_path}",
            "dest_endpoint": dest_endpoint,
            "dest_path": f"{dest_input_folder_path}",

            "result_path": f"{dest_output_folder_path}",#f"{dest_resultpath}/out_{src_filename}",
            "source_result_path": f"{src_output_folder_path}", #/out_{src_filename}",
            "fx_ep": f"{theta_fx_endpoint}",
            "fx_id": f"{func_ptycho_uuid}",
            "params": {'ifpath': f"{dest_input_folder_path}",
                       'ofpath': f"{dest_output_folder_path}/",
                       'script_path': script_path,
                       'python_path': python_path,
                       'rec_alg': rec_alg,
                       'rec_nmodes': rec_nmodes,
                       'rec_upd_pos': rec_upd_pos,
                       'rec_niter': rec_niter,
                       'rec_output_freq': rec_output_freq,
                       'rec_recover_psi': rec_recover_psi,
                       'rec_recover_probe': rec_recover_probe,
                       'rec_recover_positions': rec_recover_positions,
                       'rec_model': rec_model,
                       'rec_ngpu': rec_ngpu,
                       'rec_use_mpi': rec_use_mpi,
                       'rec_overwrite': rec_overwrite,
                       'rec_auto_pin': rec_auto_pin,
                       'rec_gpu_id':rec_gpu_id,
                       'max_qsize': MAX_QSIZE,
                       'dataset_name': dataset_name,
                       'wid':gcounter}
        }
    }
    gcounter=gcounter+1
    flow_inputs.append(flow_input)

#print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
#print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
#print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

In [8]:
print(len(flow_inputs))

168


In [9]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

print(flow)
#print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/713e1e08-ee00-45dc-8c07-c1873a0a5bca', 'administered_by': [], 'api_version': '1.0', 'created_at': '2021-06-02T16:45:08.030162+00:00', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input.params'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 3600}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destina

In [10]:
log_file_name = "/home/beams/TBICER/logs/funcx-ptycho-wf-{}-q{}-g{}.log".format(dataset_name, MAX_QSIZE, rec_ngpu)
logging.basicConfig(filename=log_file_name,
                    filemode='a',
                    format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

q0 = deque()
q1 = deque()

logging.info(f"Workflow information for: {log_file_name}")
logging.info(f"Deployed flow: {flow}")
nnodes = 8
nworkers_per_node = 8

counter=0
nflows = len(flow_inputs)
logging.info(f"Starting workflow {dataset_name}:{MAX_QSIZE}:{nnodes} left={nflows-counter} succeeded={counter}")

for i in range(len(flow_inputs)):
    if i<MAX_QSIZE:
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i])
        q1.append(flow_action)
        lstr=f"Flow {i} initiated and added to q1: {flow_action['action_id']}"
        logging.info(lstr)
        print(lstr)
    else: 
        q0.append(flow_inputs[i])
        lstr=f"Flow input {i} added to q0"
        logging.info(lstr)
        print(lstr)

i=-1
while len(q1)>0:
    time.sleep(3)
    i = (i+1)%len(q1)
    flow = flows_client.flow_action_status(flow_id, flow_scope, q1[i]['action_id'])
    
    lstr = f"len(q0)={len(q0)}; len(q1)={len(q1)}; i={i}"
    logging.info(lstr)
    print(lstr)
    
    lstr = f"Flow {i} status: {q1[i]['action_id']}: {flow['status']}" 
    logging.info(lstr)
    print(lstr)
    
    if flow['status'] == 'SUCCEEDED':
        logging.info(f"Flow succeeded: {q1[i]}")
        del q1[i]
        counter = counter+1
        if len(q0)>0:
            flow_input = q0.popleft()
            flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
            q1.append(flow_action)
            lstr = f"New flow initiated and added to the q1: {flow_action['action_id']}"
            logging.info(lstr)
            print(lstr)
            lstr=f"Copy from {flow_input['input']['params']['ifpath']} to {flow_input['input']['params']['ofpath']}"
            logging.info(lstr)
            print(lstr)
logging.info(f"Done workflow {dataset_name}:{MAX_QSIZE}:{nnodes} left={nflows-counter} succeeded={counter}")

Starting login with Globus Auth, press ^C to cancel.
Flow 0 initiated and added to q1: cc2ecb3e-a507-4e18-ab49-1a8a821989ff
Flow 1 initiated and added to q1: a7d27445-2670-43ce-b7df-68da4b988b72
Flow 2 initiated and added to q1: e0064ead-fa98-462a-bfc6-1888d6945e75
Flow 3 initiated and added to q1: d6c74419-eb31-412a-87e3-add1801016da
Flow 4 initiated and added to q1: c7787f0a-8ead-4593-a1b5-10ca9845c597
Flow 5 initiated and added to q1: 01e5982d-74ba-46d3-b2bc-7c8aefcedb01
Flow 6 initiated and added to q1: cbd3ed98-163e-4df3-bbe6-334c341fdac3
Flow 7 initiated and added to q1: d824cb01-42d3-4d6f-a819-57efeb4e9a5c
Flow 8 initiated and added to q1: 1bbc7fbf-b46d-45be-9fd0-2c4e02f67be9
Flow 9 initiated and added to q1: 49dacfd8-f453-494b-bf53-5317184d15fb
Flow 10 initiated and added to q1: 1cf60bdb-6568-4f65-b9f3-da78369d4500
Flow 11 initiated and added to q1: 13af9c33-68a0-4f6c-9a38-3b39625ef2d3
Flow 12 initiated and added to q1: c36be9fe-a86d-4775-b07c-f3ad1529071e
Flow 13 initiated and

New flow initiated and added to the q1: f0f481ad-7daf-4d22-8994-4c38fbfbed88
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/259 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/259/
len(q0)=101; len(q1)=64; i=4
Flow 4 status: d824cb01-42d3-4d6f-a819-57efeb4e9a5c: SUCCEEDED
New flow initiated and added to the q1: 10ffc4ca-14a4-4609-a54c-ac60c094f87e
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/260 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/260/
len(q0)=100; len(q1)=64; i=5
Flow 5 status: 49dacfd8-f453-494b-bf53-5317184d15fb: ACTIVE
len(q0)=100; len(q1)=64; i=6
Flow 6 status: 1cf60bdb-6568-4f65-b9f3-da78369d4500: SUCCEEDED
New flow initiated and added to the q1: 53913ee4-c9f5-489c-bf66-6f546026965f
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/261 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/261/
len(q0)=99; len(q1)=64; i=7
Flow 7 status: c36be9

len(q0)=84; len(q1)=64; i=51
Flow 51 status: d3b66f1c-5be1-4b47-bde2-f4401b2df718: ACTIVE
len(q0)=84; len(q1)=64; i=52
Flow 52 status: 6bc04c70-eb49-43d7-9ae6-4867095f0c33: ACTIVE
len(q0)=84; len(q1)=64; i=53
Flow 53 status: 879c2d05-ba7a-4e61-a45c-45503141b9c2: ACTIVE
len(q0)=84; len(q1)=64; i=54
Flow 54 status: b73886bb-828c-402b-9006-a3fb23613505: ACTIVE
len(q0)=84; len(q1)=64; i=55
Flow 55 status: 9c4f54ca-df04-49af-b7a2-d7de45be59e8: ACTIVE
len(q0)=84; len(q1)=64; i=56
Flow 56 status: f1a9b7b2-8aef-4f0f-b1c0-e183f5e707df: ACTIVE
len(q0)=84; len(q1)=64; i=57
Flow 57 status: efd729e8-887d-4941-b82b-0092c770eaa4: ACTIVE
len(q0)=84; len(q1)=64; i=58
Flow 58 status: cbb53df9-b7b5-495b-a6ae-a2a2aa29ef67: ACTIVE
len(q0)=84; len(q1)=64; i=59
Flow 59 status: 1ecf3b7d-ba41-4910-8a5d-6bce54338e2d: ACTIVE
len(q0)=84; len(q1)=64; i=60
Flow 60 status: 438cfc81-a0be-4aab-80db-d78e626017f5: ACTIVE
len(q0)=84; len(q1)=64; i=61
Flow 61 status: a0459fbb-529c-4bc0-8f36-78de4dea226a: ACTIVE
len(q0)=84

New flow initiated and added to the q1: 488509f1-be61-482a-b8ce-a4186d51ee2b
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/299 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/299/
len(q0)=61; len(q1)=64; i=23
Flow 23 status: 9d9baa28-e201-453d-b691-21c41d27793d: SUCCEEDED
New flow initiated and added to the q1: 2b258c14-e392-400c-8794-de27afef83fb
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/300 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/300/
len(q0)=60; len(q1)=64; i=24
Flow 24 status: f0f481ad-7daf-4d22-8994-4c38fbfbed88: SUCCEEDED
New flow initiated and added to the q1: ccf2e410-8715-4cc7-8426-795245e7aae5
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/301 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/301/
len(q0)=59; len(q1)=64; i=25
Flow 25 status: e1213f4f-c3ba-4849-aa4a-b9f355b39c32: SUCCEEDED
New flow initiated and added to the q1: afb

New flow initiated and added to the q1: a930f779-ac74-44bc-89f3-509e581d353c
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/317 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/317/
len(q0)=43; len(q1)=64; i=5
Flow 5 status: 97dfcaa0-e005-49ab-ba26-1ec67f62e86a: SUCCEEDED
New flow initiated and added to the q1: be07ab54-bc2a-4f68-8284-c88fc097a2d3
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/318 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/318/
len(q0)=42; len(q1)=64; i=6
Flow 6 status: ebfb1dfb-e66a-497f-89df-7151360689b6: SUCCEEDED
New flow initiated and added to the q1: 8b9c752c-4144-45cb-808c-213b712546cb
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/319 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/319/
len(q0)=41; len(q1)=64; i=7
Flow 7 status: 18e440e8-bd3e-4749-a7e0-4116cb1d5a5e: SUCCEEDED
New flow initiated and added to the q1: 345de09f-

New flow initiated and added to the q1: 03bbb3ad-2798-4741-a334-95001778c97f
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/343 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/343/
len(q0)=17; len(q1)=64; i=31
Flow 31 status: 60c1b6b5-a0ae-4bf5-842e-66f2aee81e93: SUCCEEDED
New flow initiated and added to the q1: 586d4ed7-c985-4386-b026-5b2659f46a04
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/344 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/344/
len(q0)=16; len(q1)=64; i=32
Flow 32 status: 6386ee3d-cf11-4080-98b6-a772b0b6ee81: SUCCEEDED
New flow initiated and added to the q1: b02f8397-47d9-4a44-989a-a2b73e97fda6
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/345 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/345/
len(q0)=15; len(q1)=64; i=33
Flow 33 status: e559ed05-ea71-49b9-b94d-b193d1936832: SUCCEEDED
New flow initiated and added to the q1: 6b7

len(q0)=0; len(q1)=62; i=13
Flow 13 status: afb86ac3-7e7e-4fd6-b56f-a3bd74db76df: SUCCEEDED
len(q0)=0; len(q1)=61; i=14
Flow 14 status: 4fb7af3e-faad-448f-a895-0b93b2f7483b: SUCCEEDED
len(q0)=0; len(q1)=60; i=15
Flow 15 status: 79bcbf57-e6d5-4f45-bf56-1bc3aa9e23b4: SUCCEEDED
len(q0)=0; len(q1)=59; i=16
Flow 16 status: f76c6c74-1f52-4b93-a9ea-93e0a5baa0a5: SUCCEEDED
len(q0)=0; len(q1)=58; i=17
Flow 17 status: be07ab54-bc2a-4f68-8284-c88fc097a2d3: SUCCEEDED
len(q0)=0; len(q1)=57; i=18
Flow 18 status: 3e8c1885-4964-4228-b332-22ac0b0e73cf: SUCCEEDED
len(q0)=0; len(q1)=56; i=19
Flow 19 status: f0a10325-ed5a-44fb-94ea-f0ab09f4de34: SUCCEEDED
len(q0)=0; len(q1)=55; i=20
Flow 20 status: d5aa9929-76a8-4f34-82aa-14477ff57e3f: SUCCEEDED
len(q0)=0; len(q1)=54; i=21
Flow 21 status: da27d26a-b246-41a5-924c-ea66afffff8f: SUCCEEDED
len(q0)=0; len(q1)=53; i=22
Flow 22 status: d62bdc6b-0282-44da-8427-6c5f1fecb314: SUCCEEDED
len(q0)=0; len(q1)=52; i=23
Flow 23 status: 444a72aa-9a66-4d76-80a7-26e7a330aaad

In [ ]:
print(flow_scope)

In [ ]:
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])

In [ ]:
#flows_client.flow_action_cancel(flow_id, flow_scope, '5e948f48-c3c3-4328-8d37-b5d1bc82df4c')

In [ ]:
#for ga_flow_action_id in ga_flow_actions.keys():
#    flows_client.flow_action_cancel(flow_id, flow_scope, ga_flow_action_id)

In [ ]:
#st = flows_client.flow_action_status(flow_id, flow_scope, '08925f04-569f-11e7-bef8-22000b9a448b')
#print(st['status'])

In [ ]:
!pwd